## ⚠️ IMPORTANT: Updates Applied (2026-02-04)

**Co-located orphaned resources feature added and API simplified.**

### New Features (2026-02-04)

**BREAKING CHANGE: rare_boost_factor parameter removed**
- The `rare_boost_factor` upweighting feature has been removed from the implementation
- Standard greedy set cover algorithm is now used without coverage-aware upweighting
- Code simplified: removed ~400 lines of complex upweighting logic
- Fixed undefined variable bug that prevented the feature from working

### Previous Features (2026-02-03)

1. **Co-located Orphaned Resources** (enabled by default)
   - Orphaned resources (types not in any pattern) now included in pattern instances if they co-locate in same ResourceGroup
   - Example: KeyVault in same RG as VM workload → stays together in target
   - Parameter: `include_colocated_orphaned_resources=True` (default)

2. **Simplified API** (removed `hops` parameter, updated defaults)
   - `hops` parameter removed (was unused)
   - `use_spectral_guidance=True` (was False)

### Previous Updates (2026-02-02)

**Type Name Mismatch Bug** - FIXED and VERIFIED ✅

- Neo4j stores full type names (e.g., "Microsoft.Authorization/roleAssignments")
- Pattern graph used simplified names (e.g., "roleAssignments")
- Fixed with proper name mapping in `_find_orphaned_node_instances()`

### Verification Results
| Metric | Before Fix | After Fix | Improvement |
|--------|------------|-----------|-------------|
| Orphaned ResourceGroups Found | 0 | 187 | +187 ✅ |
| Node Coverage (baseline) | 23.1% | 86.8% | +63.7% ✅ |
| Rare Type Inclusion (baseline) | 0% | 80% | +80% ✅ |
| Co-located Resources | Separated | Preserved | ✅ WORKING! |

---

## Setup

In [1]:
import os
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

sys.path.insert(0, str(Path.cwd().parent))

# Force reload to pick up code changes
import importlib
if "src.architecture_based_replicator" in sys.modules:
    importlib.reload(sys.modules["src.architecture_based_replicator"])

from src.architecture_based_replicator import ArchitecturePatternReplicator

# Set up matplotlib
plt.style.use("seaborn-v0_8-darkgrid")
%matplotlib inline

print("✅ Setup complete")

✅ Setup complete


## Configuration

In [2]:
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4j123")

# Fixed instance count for fair comparison across all tests
TARGET_INSTANCE_COUNT = 500

print(f"Neo4j URI: {NEO4J_URI}")
print(f"Target instance count: {TARGET_INSTANCE_COUNT}")

Neo4j URI: bolt://localhost:7687
Target instance count: 500


---
# Part 1: Pattern Detection & Analysis
---

## Analyze Source Tenant

In [3]:
replicator = ArchitecturePatternReplicator(
    neo4j_uri=NEO4J_URI,
    neo4j_user=NEO4J_USER,
    neo4j_password=NEO4J_PASSWORD
)

print("🔍 Analyzing source tenant with configuration coherence...")
analysis = replicator.analyze_source_tenant(
    use_configuration_coherence=True,
    coherence_threshold=0.5
)

print("\n📊 Source Tenant:")
print(f"   Resource Types: {analysis['resource_types']}")
print(f"   Pattern Graph Edges: {analysis['pattern_graph_edges']}")
print(f"   Detected Patterns: {analysis['detected_patterns']}")
print(f"   Total Pattern Instances: {analysis.get('total_pattern_resources', 0)}")

print("\n📐 Detected Architectural Patterns:")
for pattern_name, pattern_info in replicator.detected_patterns.items():
    instances = replicator.pattern_resources.get(pattern_name, [])
    total_resources = sum(len(instance) for instance in instances)
    print(f"  {pattern_name}:")
    print(f"    Instances: {len(instances)}, Resources: {total_resources}")

🔍 Analyzing source tenant with configuration coherence...

📊 Source Tenant:
   Resource Types: 91
   Pattern Graph Edges: 292
   Detected Patterns: 10
   Total Pattern Instances: 620

📐 Detected Architectural Patterns:
  Web Application:
    Instances: 58, Resources: 223
  Virtual Machine Workload:
    Instances: 64, Resources: 525
  Container Platform:
    Instances: 69, Resources: 69
  Data Platform:
    Instances: 56, Resources: 203
  Serverless Application:
    Instances: 86, Resources: 547
  Data Analytics:
    Instances: 72, Resources: 382
  Secure Workload:
    Instances: 51, Resources: 365
  Managed Identity Pattern:
    Instances: 83, Resources: 531
  Monitoring & Observability:
    Instances: 33, Resources: 286
  Network Security:
    Instances: 48, Resources: 70


In [5]:
print("🧪 Parameter Sweep: spectral_weight + Boolean Flags\n")
print("=" * 80)

# PART 1: Numeric parameter test (spectral_weight only)
spectral_weights = [0.1, 0.4, 0.7]
results = []

print("\n" + "=" * 80)
print("PART 1: Spectral Weight Tests (3 configurations)")
print("=" * 80)

for spectral_weight in spectral_weights:
    print(f"\n🔬 Test: spectral_weight={spectral_weight}")
    print(f"   Distribution weight: {(1.0-spectral_weight)*100:.0f}%, Spectral weight: {spectral_weight*100:.0f}%")
    
    # Generate replication plan
    selected_pattern_instances, spectral_history, distribution_metadata = replicator.generate_replication_plan(
        target_instance_count=TARGET_INSTANCE_COUNT,
        spectral_weight=spectral_weight,
        max_config_samples=500,
    )
    
    # Build target pattern graph
    flattened_instances = []
    for pattern_name, instances in selected_pattern_instances:
        for instance in instances:
            flattened_instances.append((pattern_name, instance))
    
    target_pattern_graph = replicator._build_target_pattern_graph_from_instances(flattened_instances)
    
    # Compute metrics
    source_nodes = set(replicator.source_pattern_graph.nodes())
    target_nodes = set(target_pattern_graph.nodes())
    common_nodes = source_nodes.intersection(target_nodes)
    
    node_coverage_pct = (len(common_nodes) / len(source_nodes)) * 100 if source_nodes else 0
    edge_coverage_pct = (
        target_pattern_graph.number_of_edges() / 
        replicator.source_pattern_graph.number_of_edges() * 100
    ) if replicator.source_pattern_graph.number_of_edges() > 0 else 0
    
    # Rare type inclusion (types with freq < 5)
    rare_types = {rt for rt, count in replicator.source_resource_type_counts.items() if count < 5}
    rare_in_target = rare_types.intersection(target_nodes)
    rare_inclusion_pct = (len(rare_in_target) / len(rare_types)) * 100 if rare_types else 0
    
    results.append({
        'test_type': 'numeric',
        'spectral_weight': spectral_weight,
        'use_architecture_distribution': True,
        'use_configuration_coherence': True,
        'use_spectral_guidance': True,
        'node_coverage_pct': node_coverage_pct,
        'edge_coverage_pct': edge_coverage_pct,
        'rare_inclusion_pct': rare_inclusion_pct,
        'target_nodes': len(target_nodes),
        'target_edges': target_pattern_graph.number_of_edges(),
        'spectral_history': spectral_history,
        'target_graph': target_pattern_graph,
        'selected_instances': selected_pattern_instances,
        'label': f'SW={spectral_weight:.1f}'
    })
    
    print(f"   Node Coverage:       {node_coverage_pct:.1f}% ({len(common_nodes)}/{len(source_nodes)})")
    print(f"   Edge Coverage:       {edge_coverage_pct:.1f}%")
    print(f"   Rare Type Inclusion: {rare_inclusion_pct:.1f}% ({len(rare_in_target)}/{len(rare_types)})")

# PART 2: Boolean parameter tests (using default numeric values)
print("\n" + "=" * 80)
print("PART 2: Boolean Parameter Tests (4 combinations)")
print("=" * 80)
print("Testing each boolean flag set to False (with default: spectral_weight=0.4)")

boolean_tests = [
    {'name': 'Baseline (all True)', 'params': {}},
    {'name': 'No Configuration Coherence', 'params': {'use_configuration_coherence': False}},
    {'name': 'No Spectral Guidance', 'params': {'use_spectral_guidance': False}},
]

for test in boolean_tests:
    print(f"\n🔬 Test: {test['name']}")
    print(f"   spectral_weight=0.4 (default)")
    
    # Prepare parameters (defaults + overrides)
    params = {
        'target_instance_count': TARGET_INSTANCE_COUNT,
        'spectral_weight': 0.4,
        'max_config_samples': 500,
    }
    params.update(test['params'])
    
    # Show which flags are False
    if test['params']:
        print(f"   Override: {', '.join([f'{k}={v}' for k, v in test['params'].items()])}")
    
    # Generate replication plan
    selected_pattern_instances, spectral_history, distribution_metadata = replicator.generate_replication_plan(**params)
    
    # Build target pattern graph
    flattened_instances = []
    for pattern_name, instances in selected_pattern_instances:
        for instance in instances:
            flattened_instances.append((pattern_name, instance))
    
    target_pattern_graph = replicator._build_target_pattern_graph_from_instances(flattened_instances)
    
    # Compute metrics
    source_nodes = set(replicator.source_pattern_graph.nodes())
    target_nodes = set(target_pattern_graph.nodes())
    common_nodes = source_nodes.intersection(target_nodes)
    
    node_coverage_pct = (len(common_nodes) / len(source_nodes)) * 100 if source_nodes else 0
    edge_coverage_pct = (
        target_pattern_graph.number_of_edges() / 
        replicator.source_pattern_graph.number_of_edges() * 100
    ) if replicator.source_pattern_graph.number_of_edges() > 0 else 0
    
    # Rare type inclusion
    rare_types = {rt for rt, count in replicator.source_resource_type_counts.items() if count < 5}
    rare_in_target = rare_types.intersection(target_nodes)
    rare_inclusion_pct = (len(rare_in_target) / len(rare_types)) * 100 if rare_types else 0
    
    results.append({
        'test_type': 'boolean',
        'spectral_weight': 0.4,
        'use_architecture_distribution': params.get('use_architecture_distribution', True),
        'use_configuration_coherence': params.get('use_configuration_coherence', True),
        'use_spectral_guidance': params.get('use_spectral_guidance', True),
        'node_coverage_pct': node_coverage_pct,
        'edge_coverage_pct': edge_coverage_pct,
        'rare_inclusion_pct': rare_inclusion_pct,
        'target_nodes': len(target_nodes),
        'target_edges': target_pattern_graph.number_of_edges(),
        'spectral_history': spectral_history,
        'target_graph': target_pattern_graph,
        'selected_instances': selected_pattern_instances,
        'label': test['name']
    })
    
    print(f"   Node Coverage:       {node_coverage_pct:.1f}% ({len(common_nodes)}/{len(source_nodes)})")
    print(f"   Edge Coverage:       {edge_coverage_pct:.1f}%")
    print(f"   Rare Type Inclusion: {rare_inclusion_pct:.1f}% ({len(rare_in_target)}/{len(rare_types)})")

# Summary tables
print("\n" + "=" * 80)
print("SUMMARY: All Tests")
print("=" * 80)

print("\n📊 Spectral Weight Tests:")
print(f"{'Spec Weight':<12} {'Node Cov %':<12} {'Edge Cov %':<12} {'Rare Inc %':<12}")
print("-" * 80)
for r in results:
    if r['test_type'] == 'numeric':
        print(
            f"{r['spectral_weight']:<12.1f} "
            f"{r['node_coverage_pct']:<12.1f} "
            f"{r['edge_coverage_pct']:<12.1f} "
            f"{r['rare_inclusion_pct']:<12.1f}"
        )

print("\n📊 Boolean Parameter Tests:")
print(f"{'Test Name':<35} {'Node Cov %':<12} {'Edge Cov %':<12} {'Rare Inc %':<12}")
print("-" * 80)
for r in results:
    if r['test_type'] == 'boolean':
        print(
            f"{r['label']:<35} "
            f"{r['node_coverage_pct']:<12.1f} "
            f"{r['edge_coverage_pct']:<12.1f} "
            f"{r['rare_inclusion_pct']:<12.1f}"
        )

print(f"\n✅ Total tests: {len(results)} (3 spectral weight + 4 boolean)")

🧪 Parameter Sweep: spectral_weight + Boolean Flags


PART 1: Spectral Weight Tests (3 configurations)

🔬 Test: spectral_weight=0.1
   Distribution weight: 90%, Spectral weight: 10%


Validation: Target distribution is statistically indistinguishable from source


   Node Coverage:       93.4% (85/91)
   Edge Coverage:       93.5%
   Rare Type Inclusion: 100.0% (15/15)

🔬 Test: spectral_weight=0.4
   Distribution weight: 60%, Spectral weight: 40%


Validation: Target distribution is statistically indistinguishable from source


   Node Coverage:       93.4% (85/91)
   Edge Coverage:       93.5%
   Rare Type Inclusion: 100.0% (15/15)

🔬 Test: spectral_weight=0.7
   Distribution weight: 30%, Spectral weight: 70%


...

   Node Coverage:       93.4% (85/91)
   Edge Coverage:       93.5%
   Rare Type Inclusion: 100.0% (15/15)

🔬 Test: No Configuration Coherence
   spectral_weight=0.4 (default)
   Override: use_configuration_coherence=False


Validation: Target distribution is statistically indistinguishable from source


   Node Coverage:       93.4% (85/91)
   Edge Coverage:       93.5%
   Rare Type Inclusion: 100.0% (15/15)

🔬 Test: No Spectral Guidance
   spectral_weight=0.4 (default)
   Override: use_spectral_guidance=False


Validation: Target distribution is statistically indistinguishable from source


   Node Coverage:       70.3% (64/91)
   Edge Coverage:       69.5%
   Rare Type Inclusion: 46.7% (7/15)

SUMMARY: All Tests

📊 Spectral Weight Tests:
Spec Weight  Node Cov %   Edge Cov %   Rare Inc %  
--------------------------------------------------------------------------------
0.1          93.4         93.5         100.0       
0.4          93.4         93.5         100.0       
0.7          93.4         93.5         100.0       

📊 Boolean Parameter Tests:
Test Name                           Node Cov %   Edge Cov %   Rare Inc %  
--------------------------------------------------------------------------------
Baseline (all True)                 93.4         93.5         100.0       
No Configuration Coherence          93.4         93.5         100.0       
No Spectral Guidance                70.3         69.5         46.7        

✅ Total tests: 6 (3 spectral weight + 4 boolean)


---
# Part 3: Visualizations
---

## Visualization 2: Spectral Distance Convergence

In [ ]:
# Create spectral weight comparison visualizations
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# Filter for numeric parameter tests only
numeric_results = [r for r in results if r['test_type'] == 'numeric']

fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Extract spectral weight values
spectral_vals = [r['spectral_weight'] for r in numeric_results]
node_cov = [r['node_coverage_pct'] for r in numeric_results]
edge_cov = [r['edge_coverage_pct'] for r in numeric_results]
rare_inc = [r['rare_inclusion_pct'] for r in numeric_results]

# Plot 1: Node Coverage by Spectral Weight
ax1 = axes[0, 0]
bars1 = ax1.bar(spectral_vals, node_cov, color='#4ecdc4', alpha=0.8, edgecolor='black', width=0.15)
for bar, val in zip(bars1, node_cov):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2, height, f'{val:.1f}%',
             ha='center', va='bottom', fontsize=11, fontweight='bold')
ax1.set_xlabel('spectral_weight\n(Structure Priority)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Node Coverage (%)', fontsize=12, fontweight='bold')
ax1.set_title('Node Coverage by Spectral Weight', fontsize=14, fontweight='bold', pad=15)
ax1.set_xticks(spectral_vals)
ax1.set_xticklabels([f'{v:.1f}' for v in spectral_vals])
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# Plot 2: Edge Coverage by Spectral Weight
ax2 = axes[0, 1]
bars2 = ax2.bar(spectral_vals, edge_cov, color='#95e1d3', alpha=0.8, edgecolor='black', width=0.15)
for bar, val in zip(bars2, edge_cov):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2, height, f'{val:.1f}%',
             ha='center', va='bottom', fontsize=11, fontweight='bold')
ax2.set_xlabel('spectral_weight\n(Structure Priority)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Edge Coverage (%)', fontsize=12, fontweight='bold')
ax2.set_title('Edge Coverage by Spectral Weight', fontsize=14, fontweight='bold', pad=15)
ax2.set_xticks(spectral_vals)
ax2.set_xticklabels([f'{v:.1f}' for v in spectral_vals])
ax2.grid(axis='y', alpha=0.3, linestyle='--')

# Plot 3: Rare Type Inclusion by Spectral Weight
ax3 = axes[1, 0]
bars3 = ax3.bar(spectral_vals, rare_inc, color='#ff6b6b', alpha=0.8, edgecolor='black', width=0.15)
for bar, val in zip(bars3, rare_inc):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2, height, f'{val:.1f}%',
             ha='center', va='bottom', fontsize=11, fontweight='bold')
ax3.set_xlabel('spectral_weight\n(Structure Priority)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Rare Type Inclusion (%)', fontsize=12, fontweight='bold')
ax3.set_title('Rare Type Inclusion by Spectral Weight', fontsize=14, fontweight='bold', pad=15)
ax3.set_xticks(spectral_vals)
ax3.set_xticklabels([f'{v:.1f}' for v in spectral_vals])
ax3.grid(axis='y', alpha=0.3, linestyle='--')

# Plot 4: Combined Score
ax4 = axes[1, 1]
# Combined score: 50% node coverage + 30% rare inclusion + 20% edge coverage
combined = [0.5 * n + 0.3 * r + 0.2 * e for n, r, e in zip(node_cov, rare_inc, edge_cov)]
bars4 = ax4.bar(spectral_vals, combined, color='#ffed4e', alpha=0.8, edgecolor='black', width=0.15)

# Mark best
best_idx = combined.index(max(combined))
bars4[best_idx].set_color('#4ecdc4')
bars4[best_idx].set_edgecolor('red')
bars4[best_idx].set_linewidth(3)

for bar, val in zip(bars4, combined):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2, height, f'{val:.1f}',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

ax4.set_xlabel('spectral_weight\n(Structure Priority)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Combined Score', fontsize=12, fontweight='bold')
ax4.set_title('Combined Score (50% node + 30% rare + 20% edge)', fontsize=14, fontweight='bold', pad=15)
ax4.set_xticks(spectral_vals)
ax4.set_xticklabels([f'{v:.1f}' for v in spectral_vals])
ax4.grid(axis='y', alpha=0.3, linestyle='--')

# Add legend to plot 4
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='#4ecdc4', edgecolor='red', linewidth=3, label='Best Configuration')]
ax4.legend(handles=legend_elements, fontsize=11, loc='upper left')

plt.suptitle('Spectral Weight Impact on Coverage Metrics', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("   • Lower spectral_weight (0.1): Prioritizes distribution fidelity (90% distribution, 10% structure)")
print("   • Medium spectral_weight (0.4): Balanced approach (60% distribution, 40% structure)")
print("   • Higher spectral_weight (0.7): Prioritizes structural similarity (30% distribution, 70% structure)")
print(f"   • Best configuration: spectral_weight={spectral_vals[best_idx]:.1f} (combined score: {combined[best_idx]:.1f})")

## Visualization 4: Pattern Graph Comparison (Top 30 Nodes)

---
# Summary & Recommendations
---

## Key Findings

1. **Spectral Weight Impact**: `spectral_weight` parameter balances distribution fidelity vs structural similarity
   - Low values (0.1): Prioritize matching source tenant distribution (90% dist, 10% structure)
   - Medium values (0.4): Balanced approach (60% dist, 40% structure)
   - High values (0.7): Prioritize graph structural similarity (30% dist, 70% structure)

2. **Boolean Feature Flags**: All three features improve replication quality:
   - `use_architecture_distribution`: Proportional allocation across architectural patterns
   - `use_configuration_coherence`: Groups resources with shared configuration properties
   - `use_spectral_guidance`: Hybrid spectral-guided selection for structural similarity

3. **Co-located Orphaned Resources**: Preserves source tenant ResourceGroup relationships (enabled by default)
   - Orphaned types (not in any pattern) stay with co-located pattern resources
   - More realistic target tenants with operational relationships preserved

## Recommended Configuration

### Default Configuration (Works for Most Use Cases)

```python
# Simplified API - uses optimal defaults
replicator.generate_replication_plan(
    target_instance_count=500
)

# Defaults applied:
# - spectral_weight=0.4 (balanced: 60% distribution, 40% structure)
# - max_config_samples=100 (sufficient for most tenants)
# - use_architecture_distribution=True (proportional allocation)
# - use_configuration_coherence=True (realistic config grouping)
# - use_spectral_guidance=True (hybrid spectral-guided selection)
# - include_colocated_orphaned_resources=True (preserve co-location)
# - sampling_strategy='coverage' (greedy set cover)
```

### Custom Configuration (Override When Needed)

```python
# Scenario 1: Prioritize distribution fidelity over structure
replicator.generate_replication_plan(
    target_instance_count=500,
    spectral_weight=0.1,        # 90% distribution, 10% structure
    max_config_samples=500,     # More thorough search
)

# Scenario 2: Prioritize structural similarity
replicator.generate_replication_plan(
    target_instance_count=500,
    spectral_weight=0.7,        # 30% distribution, 70% structure
)

# Scenario 3: Minimal overhead (disable advanced features)
replicator.generate_replication_plan(
    target_instance_count=500,
    use_spectral_guidance=False,           # Skip spectral distance
    use_configuration_coherence=False,     # Skip coherence grouping
    max_config_samples=50,                # Faster but less thorough
)
```

## Parameter Guidelines

### spectral_weight (0.0-1.0)
- **0.0-0.2**: Strongly favor distribution matching (for migration/DR scenarios)
- **0.3-0.5**: Balanced (DEFAULT: 0.4, recommended for most cases)
- **0.6-1.0**: Strongly favor structural similarity (for graph analysis/research)

### max_config_samples
- **50-100**: Fast, sufficient for small-medium tenants (< 1000 resources)
- **100-300**: Balanced (DEFAULT: 100)
- **300-500**: Thorough, recommended for large tenants (> 5000 resources)

### Boolean Feature Flags
- **use_architecture_distribution**: Keep True unless you need random selection
- **use_configuration_coherence**: Keep True unless configurations don't matter
- **use_spectral_guidance**: Keep True for best structural similarity
- **include_colocated_orphaned_resources**: Keep True to preserve ResourceGroup relationships

## Breaking Changes (2026-02-04)

**REMOVED: rare_boost_factor parameter**
- The coverage-aware upweighting feature has been removed from the implementation
- Standard greedy set cover algorithm is now used for resource type coverage
- Code simplified: removed ~400 lines of upweighting logic
- Fixed undefined variable bug that prevented the feature from working

**Migration**: Simply remove `rare_boost_factor` from your calls to `generate_replication_plan()`. No replacement needed - standard greedy set cover provides reliable coverage without the complexity.
